El Excel que compartiremos con los seminaristas tiene unos enlaces de alquiler y venta que fueron descargados entre marzo y abril de 2024. Algunos anuncios ya han sido retirados, y estos, al abrir sus enlaces, te redireccionan hacia otro enlace.

Considerando esto, he tratado de identificarlos para retirarlos y disminuir el trabajo de los seminaristas.


In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import requests
from urllib.parse import urlparse

In [17]:
df=pd.read_excel('/content/drive/MyDrive/Documentos personales/9-Documentos Doctorado/Doctorado 2023/7-Clases/SRGVUA/evacal/prueba_enlaces_invalidos.xlsx')

In [ ]:
df.head()

,Link,precio_euros,Description,area,n_rooms,n_baths,terrace,consum_A,consum_B,consum_C,...,lift,NOM,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,https://www.habitaclia.com/comprar-piso-poblen...,189900,"697,73€* Cuota mensual hipoteca a tipo fijo. L...",51,2,1,0,0,0,0,...,1,la Vila OlÃ­mpica del Poblenou,38.5,1.0,5.0,5.0,59669.0,235681.813228,-45781.813228,1.0
1,https://www.habitaclia.com/comprar-piso-oportu...,300000,Bienvenido a Grup Cuatrecaps! Déjanos acompaña...,95,4,1,1,0,0,0,...,1,el Clot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


El primer enlace es inválido; el segundo, válido.

In [ ]:
invalido=df.loc[0, "Link"]
valido=df.loc[1, "Link"]

Si aplicamos `requests` para entrar al enlace y ver la redirección, obtengo lo siguiente: el enlace inválido , si le damos clic, nos redirecciona hacia "https://www.habitaclia.com/viviendas-barcelona.htm?state=1", pero según `requests`, el enlace sigue siendo el mismo, lo cual es un error.

In [ ]:
respuesta = requests.head(valido, allow_redirects=True)
url_redireccionada = respuesta.url
print(url_redireccionada)

https://www.habitaclia.com/comprar-piso-oportunidad_grande_de_4_habitaciones_con_balcon_en_plena_ra_el_clot-barcelona-i45207000000035.htm?pag=1001&f=&geo=c&from=list&lo=55


In [ ]:
respuesta = requests.head(invalido, allow_redirects=True)
url_redireccionada = respuesta.url
print(url_redireccionada)

https://www.habitaclia.com/comprar-piso-poblenou_la_vila_olimpica_del_poblenou-barcelona-i3118004333089.htm?pag=413&f=&geo=c&from=list&lo=55


Pero usando `head` y una solicitud 'user-agent' se soluciona.

In [15]:
requests.head(invalido, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})

<Response [301]>

In [16]:
requests.head(valido, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})

<Response [200]>

Ahora creo una función para aplicarla a todo el df.

In [18]:
# Función para verificar si un enlace es válido
def verificar_enlace_valido(link):
    try:
        # Realizamos una solicitud GET al enlace
        response = requests.head(link, headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})
        # Si la solicitud es exitosa (código de estado 200), consideramos el enlace como válido
        if response.status_code == 200:
            return 0  # Enlace válido
        else:
            return 1  # Enlace inválido
    except:
        return 1  # En caso de algún error al abrir el enlace, lo marcamos como inválido

# Aplicamos la función a la columna 'link' y guardamos los resultados en una nueva columna 'invalido'
df['invalido'] = df['Link'].apply(verificar_enlace_valido)

df['invalido'].head()

0    1
1    0
Name: invalido, dtype: int64